# Predicting tags for the given code
### Using simple models (SVM, LogReg, Naive Bayes, LGBM)

In [ ]:
# !pip install dagshub

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC

import pickle
import dagshub

In [ ]:
def load_code_blocks(DATASET_PATH, CODE_COLUMN):
    df = pd.read_csv(DATASET_PATH, encoding='utf-8', comment='#', sep='\t')#, quoting=csv.QUOTE_NONE, error_bad_lines=False)#, sep=','
    print("Data of {} shape has been loaded".format(df.shape))
    print(df.columns)
    code_blocks = df[CODE_COLUMN]
    # test_size = 0.1
    # test_rows = round(df.shape[0]*test_size)
    # train_rows = df.shape[0] - test_rows
    # train_code_blocks = df[CODE_COLUMN][0:test_rows]
    # test_code_blocks = df[CODE_COLUMN][train_rows:]
    return df, code_blocks

In [ ]:
def tfidf_fit_transform(code_blocks, params, TFIDF_DIR):
    vectorizer = TfidfVectorizer(params)
    tfidf = vectorizer.fit(code_blocks)
    pickle.dump(tfidf, open("tfidf.pickle", "wb"))
    code_blocks_tfidf = tfidf.transform(code_blocks)
    print("tfidf trained, saved and transformed")
    return code_blocks_tfidf

In [ ]:
def tfidf_transform(code_blocks, params, TFIDF_DIR):
    tfidf = pickle.load(open(TFIDF_DIR, 'rb'))
    code_blocks_tfidf = tfidf.transform(code_blocks)
    print("tfidf loaded and transformed")
    return code_blocks_tfidf

In [ ]:
def SVM_evaluate(df, code_blocks, tfidf_params, TFIDF_DIR, SVM_params):
    code_blocks_tfidf = tfidf_fit_transform(code_blocks, tfidf_params, TFIDF_DIR)
    X_train, X_test, y_train, y_test = train_test_split(code_blocks_tfidf, df[TAG_TO_PREDICT], test_size=0.3)
    # grid = {"C": [100]}
    # cv = KFold(n_splits=2, shuffle=True, random_state=241)
    # model = SVC(kernel="linear", random_state=241)
    # gs = GridSearchCV(model, grid, scoring="accuracy", cv=cv, verbose=1, n_jobs=-1)
    # gs.fit(X_train[:25000], y_train.ravel()[:25000])
    # C = gs.best_params_.get('C')
    model = SVC(**SVM_params)
    # model.set_params(SVM_params)
    print(model.get_params())
    model.fit(X_train, y_train.ravel())
    # a faster option:
    # 1: usage of BagginClassifier decreased the fitting time from 38 mins to 8
    # n_estimators = 10
    clf = model
    # clf = BaggingClassifier(model, max_samples=1.0 / n_estimators, n_estimators=n_estimators)
    # clf.fit(X_train, y_train.ravel())
    pickle.dump(clf, open(MODEL_DIR, 'wb'))
    # y_pred = model.predict(X_test)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    metrics = {'test_accuracy': accuracy
               , 'test_f1_score': f1}
    plot_confusion_matrix(model, X_test, y_test)
    return metrics

In [ ]:
def get_predictions(X, y, TAG_TO_PREDICT, MODEL_DIR):
    clf = pickle.load(open(MODEL_DIR, 'rb'))
    print("the model has been loaded")
    # result = loaded_model.score(X, y)
    y_pred = clf.predict(X)
    print("predictions were calculated")
    accuracy = clf.score(X, y)
    f1 = f1_score(y_pred, y, average='weighted')
    print(f'Mean Accuracy {round(accuracy*100, 2)}%')
    print(f'F1-score {round(f1*100, 2)}%')
    errors = y - y_pred
    plt.hist(errors)
    plot_precision_recall_curve(clf, X, y)
    plot_confusion_matrix(clf, X, y, values_format='d')
    def mean_confidence_interval(data, confidence=0.95):
        a = 1.0 * np.array(data)
        n = len(a)
        m, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        return m, m-h, m+h
    conf_interval = mean_confidence_interval(errors, 0.95)
    print(conf_interval)
    metrics = {'test_accuracy': accuracy
               , 'test_f1_score': f1
               , 'conf_interval': conf_interval}
    return X, y, y_pred, metrics
    print(result)

### Predictions and Logging

In [ ]:
# if __name__ == '__main__':
#     DATASET_PATH = './data/chunks_5_validate.csv'
#     MODEL_DIR = './models/svm_regex_graph_v2.1.sav'
#     TFIDF_DIR = './models/tfidf_svm_graph_v2.1.pickle'
#     CODE_COLUMN = 'code' #'code_block'
#     TAG_TO_PREDICT = 'tag' #'preprocessing'
#     SCRIPT_DIR = 'predict_tag.ipynb'

#     df, code_blocks = load_code_blocks(DATASET_PATH, CODE_COLUMN)
#     nrows = df.shape[0]
#     print("loaded")
#     tfidf_params = {'min_df': 5
#             , 'max_df': 0.3
#             , 'smooth_idf': True}
#     SVM_params = {'C': 100
#             , 'kernel': "linear"
#             , 'verbose': 1
#             , 'random_state': 241}
#     meta = {'data': DATASET_PATH
#                 ,'nrows': nrows
#                 ,'label': TAG_TO_PREDICT
#                 ,'model': MODEL_DIR
#                 ,'script_dir': SCRIPT_DIR}

#     code_blocks_tfidf = tfidf_transform(code_blocks, tfidf_params, TFIDF_DIR)
#     with dagshub.dagshub_logger() as logger:
#         # metrics = SVM_evaluate(df, code_blocks, tfidf_params, TFIDF_DIR, SVM_params)
#         _, _, _, metrics = get_predictions(code_blocks_tfidf, df[TAG_TO_PREDICT].astype(int), TAG_TO_PREDICT, MODEL_DIR)
#         logger.log_hyperparams(meta)
#         logger.log_hyperparams(tfidf_params)
#         logger.log_hyperparams(SVM_params)
#         logger.log_metrics(metrics)
#     print("finished")

### Validation (different chunk_sizes)

In [ ]:
if __name__ == '__main__':
    all_metrics = []
    chunk_sizes = [5, 10, 15, 20, 25, 30, 40]
    for i in chunk_sizes:
        GRAPH_VER = 2.2
        MODEL = 'svm'
        DATASET_PATH = './data/chunks_{}_validate.csv'.format(i)
        MODEL_DIR = './models/{}_regex_graph_v{}.sav'.format(MODEL, GRAPH_VER)
        TFIDF_DIR = './models/tfidf_{}_graph_v{}.pickle'.format(MODEL, GRAPH_VER)
        CODE_COLUMN = 'code'
        TAG_TO_PREDICT = 'tag'
        df, corpus = load_code_blocks(DATASET_PATH, CODE_COLUMN)
        nrows = df.shape[0]
        print("loaded")
        params = {'min_df': 5
                , 'max_df': 0.3
                , 'smooth_idf': True}
        data_meta = {'DATASET_PATH': DATASET_PATH
                    ,'nrows': nrows
                    ,'label': TAG_TO_PREDICT
                    ,'model': 'Logistic Regression'}
        features = tfidf_transform(corpus, params, TFIDF_DIR)
        print("tfidf-ed")
        # metrics = logreg_evaluate(df, features, TAG_TO_PREDICT)
        _, _, _, metrics = get_predictions(features, df[TAG_TO_PREDICT], TAG_TO_PREDICT, MODEL_DIR)
        all_metrics.append(metrics)
        # with dagshub.dagshub_logger() as logger:
        #     logger.log_hyperparams(data_meta)
        #     logger.log_hyperparams(params)
        #     logger.log_metrics(metrics)
        print("finished")

In [ ]:
all_acc = [all_metrics[i]['test_accuracy'] for i in range(len(all_metrics))]
all_f1 = [all_metrics[i]['test_f1_score'] for i in range(len(all_metrics))]
plt.scatter(chunk_sizes, all_acc)
plt.scatter(chunk_sizes, all_f1)

## Errors Analysis

In [ ]:
GRAPH_VER = 2.2
MODEL = 'logreg'
CHUNK_SIZE = 10
CODE_COLUMN = 'code'
TAG_TO_PREDICT = 'tag'

DATASET_PATH = './data/chunks_{}_validate.csv'.format(CHUNK_SIZE)
MODEL_DIR = './models/{}_regex_graph_v{}.sav'.format(MODEL, GRAPH_VER)
TFIDF_DIR = './models/tfidf_{}_graph_v{}.pickle'.format(MODEL, GRAPH_VER)
df, corpus = load_code_blocks(DATASET_PATH, CODE_COLUMN)
nrows = df.shape[0]
print("loaded")
params = {'min_df': 5
        , 'max_df': 0.3
        , 'smooth_idf': True}
data_meta = {'DATASET_PATH': DATASET_PATH
            ,'nrows': nrows
            ,'label': TAG_TO_PREDICT
            ,'model': 'Logistic Regression'}
features = tfidf_transform(corpus, params, TFIDF_DIR)
print("tfidf-ed")

In [ ]:
_, y, y_pred, _ = get_predictions(features, df[TAG_TO_PREDICT], TAG_TO_PREDICT, MODEL_DIR)

In [ ]:
pd.set_option('max_colwidth', 500)
pd.set_option('max_rows', 500)

In [49]:
## False Negatives (preprocessing y/n)
fn = corpus[(y_pred-y)==-1].reset_index(drop=True)[:]
fn.to_csv('{}_validation_{}_chunksize_{}_conf_matrix_FN.txt'.format(MODEL, GRAPH_VER, CHUNK_SIZE), index=False)
# fn

In [50]:
## False Positives (preprocessing y/n)
fp = corpus[(y_pred-y)==1].reset_index(drop=True)[:]
fp.to_csv('{}_validation_{}_chunksize_{}_conf_matrix_FP.txt'.format(MODEL, GRAPH_VER, CHUNK_SIZE), index=False)
# fp

In [51]:
## True (preprocessing y/n)
t = corpus[(y_pred-y)==0].reset_index(drop=True)[:]
t.to_csv('{}_validation_{}_chunksize_{}_conf_matrix_True.txt'.format(MODEL, GRAPH_VER, CHUNK_SIZE), index=False)
# t